# Parse and Compile US Climbing Areas 
I started by downloading all zip files from [OpenBeta's climbing data repo](https://github.com/OpenBeta/climbing-data/tree/main), unzipping them, and copying all of the `-areas.jsonlines` files to the local directory. Let's use this to create a somewhat comprehensive set of climbing areas with location coordinates to populate our dropdown.

In [1]:
import pandas as pd 
import os 

I'd like to read one just to see what it's like. We'll start with Tennessee

In [2]:
tn_df = pd.read_json('raw_data/tn-areas.jsonlines', lines=True)
tn_df.shape

(310, 8)

In [3]:
tn_df.head()

,area_name,description,location,path,us_state,url,lnglat,metadata
0,Buffalo Rock on French Broad River,[Secluded rock outcropping on the sand banks o...,"[Take brush creek road until it splits, go to ...",Buffalo Rock on French Broad River,Tennessee,https://www.mountainproject.com/area/118222027...,"[-82.93149, 35.93945]",{'lnglat_from_parent': False}
1,Bells Bend,[This is an artificial boulder next to Bells B...,"[4187 Old Hickory Blvd, Nashville, TN 37218, ...",Bells Bend,Tennessee,https://www.mountainproject.com/area/119104219...,"[-86.92197, 36.15453]",{'lnglat_from_parent': False}
2,Bee Rock,[Bee Rock is considered the first developed cl...,"[Google Bee Rock Overlook, Monterey, TN. Park ...",Bee Rock,Tennessee,https://www.mountainproject.com/area/121646957...,"[-85.28619, 36.12879]",{'lnglat_from_parent': False}
3,Cherokee Bluff (CLOSED TO CLIMBING),[Cherokee Bluff is currently closed to climbin...,[This area has a handful of sport routes locat...,Cherokee Bluff (CLOSED TO CLIMBING),Tennessee,https://www.mountainproject.com/area/106768062...,"[-83.9618, 35.9247]",{'lnglat_from_parent': False}
4,Parking lot side of street,"[Usually quite cool in summer, with leaves on ...",[One minute walk from parking lot. Mostly top-...,Backbone Rock|Parking lot side of street,Tennessee,https://www.mountainproject.com/area/111134147...,"[-81.8151, 36.594]",{'lnglat_from_parent': False}


The `path` column has a pipe-delimited value that contaians the hierarchical path of `Parent|Child|Child|Child` from broadest to most granular areas. However, large crags with several sub-areas will have more granularity than random isolated areas with names like `Parking lot side of street`. I'm either going to need to just take the highest-level parent or join to routes and filter out the ones with barely anything

Let's see what option A brings

In [4]:
tn_df['area_name'] = tn_df['path'].str.split('|').str[0]
tn_df['area_name'].value_counts(dropna=False)

area_name
Obed & Clear Creek                      51
Boulders on Old Wauhatchie Pike, The    23
Foster Falls                            19
Big South Fork                          19
Denny Cove                              18
                                        ..
Nolichucky riverside                     1
Monterey Crag                            1
Mizell Bluffs on Tellico Lake            1
Lower Leda                               1
Buffalo Rock on French Broad River       1
Name: count, Length: 63, dtype: int64

Yikes... some of these make more sense than others. Either way this is definitely a better start than just using everything or hard-coding. Might be good to one day use something like OpenBeta's GraphQL API, but this is fine for now (albeit somewhat outdated)

A couple of considerations:
* There may be some closed areas
* What about some more recent areas?

In [5]:
tn_df[tn_df['area_name'].str.contains("closed", case=False)]

,area_name,description,location,path,us_state,url,lnglat,metadata
3,Cherokee Bluff (CLOSED TO CLIMBING),[Cherokee Bluff is currently closed to climbin...,[This area has a handful of sport routes locat...,Cherokee Bluff (CLOSED TO CLIMBING),Tennessee,https://www.mountainproject.com/area/106768062...,"[-83.9618, 35.9247]",{'lnglat_from_parent': False}


In [6]:
tn_df[tn_df['path'].str.contains("closed", case=False)]

,area_name,description,location,path,us_state,url,lnglat,metadata
3,Cherokee Bluff (CLOSED TO CLIMBING),[Cherokee Bluff is currently closed to climbin...,[This area has a handful of sport routes locat...,Cherokee Bluff (CLOSED TO CLIMBING),Tennessee,https://www.mountainproject.com/area/106768062...,"[-83.9618, 35.9247]",{'lnglat_from_parent': False}


We should exclude anything that includes "closed" in the title. I checked descriptions as well, but those were primarily things like seasonal closures instead of full area closures

Now let's check if Woodcock is in here

In [7]:
tn_df[tn_df['path'].str.contains("woodcock", case=False)]

,area_name,description,location,path,us_state,url,lnglat,metadata


It is not. We'll add it later

In [8]:
tn_df[['lon', 'lat']] = tn_df['lnglat'].to_list()

In [9]:
tn_df.head()

,area_name,description,location,path,us_state,url,lnglat,metadata,lon,lat
0,Buffalo Rock on French Broad River,[Secluded rock outcropping on the sand banks o...,"[Take brush creek road until it splits, go to ...",Buffalo Rock on French Broad River,Tennessee,https://www.mountainproject.com/area/118222027...,"[-82.93149, 35.93945]",{'lnglat_from_parent': False},-82.93149,35.93945
1,Bells Bend,[This is an artificial boulder next to Bells B...,"[4187 Old Hickory Blvd, Nashville, TN 37218, ...",Bells Bend,Tennessee,https://www.mountainproject.com/area/119104219...,"[-86.92197, 36.15453]",{'lnglat_from_parent': False},-86.92197,36.15453
2,Bee Rock,[Bee Rock is considered the first developed cl...,"[Google Bee Rock Overlook, Monterey, TN. Park ...",Bee Rock,Tennessee,https://www.mountainproject.com/area/121646957...,"[-85.28619, 36.12879]",{'lnglat_from_parent': False},-85.28619,36.12879
3,Cherokee Bluff (CLOSED TO CLIMBING),[Cherokee Bluff is currently closed to climbin...,[This area has a handful of sport routes locat...,Cherokee Bluff (CLOSED TO CLIMBING),Tennessee,https://www.mountainproject.com/area/106768062...,"[-83.9618, 35.9247]",{'lnglat_from_parent': False},-83.96180,35.92470
4,Backbone Rock,"[Usually quite cool in summer, with leaves on ...",[One minute walk from parking lot. Mostly top-...,Backbone Rock|Parking lot side of street,Tennessee,https://www.mountainproject.com/area/111134147...,"[-81.8151, 36.594]",{'lnglat_from_parent': False},-81.81510,36.59400


Alas, it is not. I have the coordinates for it, so I'll just include some known misses

In [10]:
TOP_LEVEL_STATES = [
    'Tennessee', 
    'California', 
    'Rhode Island', 
    'Alabama', 
    'Georgia'
]

# THIRD_LEVEL_STATES = [
#     'Utah', 
#     'Arizona'
# ]

def get_area_name(df):
    split = df.path.split('|')
    if (df.us_state in TOP_LEVEL_STATES) | (len(split) == 1):
        return split[0]
    else:
        return split[1]

In [11]:
def load_and_scrub_state_areas_df(path):
    df = pd.read_json(path, lines=True)
    if df.shape[0] > 0:
        df['area_name'] = df.apply(get_area_name, axis=1)
        df[['longitude', 'latitude']] = df['lnglat'].to_list()
        return df[['area_name', 'us_state', 'path', 'latitude', 'longitude']]

In [12]:
dfs = []

for filename in os.listdir('raw_data'):
    dfs += [load_and_scrub_state_areas_df(os.path.join(f'raw_data/{filename}'))]

full_df = pd.concat(dfs, ignore_index=True).reset_index(drop=True)
full_df.shape

(35876, 5)

In [13]:
full_df.isna().sum()

area_name    0
us_state     0
path         0
latitude     0
longitude    0
dtype: int64

Awesome

In [14]:
full_df.groupby(['area_name', 'us_state']).agg({'latitude':['count','mean'], 'longitude':'mean'})\
    .sort_values(('latitude', 'count'), ascending=False).head(25)

latitude             \
                                                         count       mean   
area_name                               us_state                            
Joshua Tree National Park               California        1288  34.023512   
Sierra Eastside                         California        1081  37.397513   
Unaweep Canyon                          Colorado           903  38.900867   
Lake Tahoe                              California         828  39.026414   
Red Rocks                               Nevada             495  36.133435   
San Diego County                        California         472  33.034560   
Cape Ann                                Massachusetts      466  42.616056   
Central Wasatch                         Utah               455  40.602752   
Devil's Lake                            Wisconsin          450  43.418145   
San Bernardino Mountains                California         424  34.264001   
Yosemite National Park                  California         422  37.773910   
Mount Lemmon (Santa Catalina Mountains) Arizona            415  32.379793   
High Desert                             California         391  34.661786   
San Jacinto Mountains                   California         382  33.807961   
Los Angeles Basin                       California         382  34.161765   
Central Coast                           California         372  35.302081   
San Francisco Bay Area                  California         359  37.963906   
Grayson Highlands State Park            Virginia           343  36.620018   
Lynn Woods                              Massachusetts      325  42.497249   
Poudre Canyon                           Colorado           295  40.675770   
**Phoenix Areas                         Arizona            275  33.515199   
Charlottesville vicinity                Virginia           249  37.979431   
Western Sierra                          California         247  36.879377   
Mount Rushmore National Memorial        South Dakota       237  43.886861   
Northeast California                    California         227  40.314169   

                                                        longitude  
                                                             mean  
area_name                               us_state                   
Joshua Tree National Park               California    -116.138074  
Sierra Eastside                         California    -118.635186  
Unaweep Canyon                          Colorado      -108.510892  
Lake Tahoe                              California    -120.370141  
Red Rocks                               Nevada        -115.453242  
San Diego County                        California    -116.743661  
Cape Ann                                Massachusetts  -70.694685  
Central Wasatch                         Utah          -111.740632  
Devil's Lake                            Wisconsin      -89.724892  
San Bernardino Mountains                California    -117.015946  
Yosemite National Park                  California    -119.538944  
Mount Lemmon (Santa Catalina Mountains) Arizona       -110.743931  
High Desert                             California    -117.044223  
San Jacinto Mountains                   California    -116.693576  
Los Angeles Basin                       California    -118.263071  
Central Coast                           California    -120.389913  
San Francisco Bay Area                  California    -122.414396  
Grayson Highlands State Park            Virginia       -81.493642  
Lynn Woods                              Massachusetts  -70.994241  
Poudre Canyon                           Colorado      -105.626038  
**Phoenix Areas                         Arizona       -111.886287  
Charlottesville vicinity                Virginia       -78.917778  
Western Sierra                          California    -119.118706  
Mount Rushmore National Memorial        South Dakota  -103.459444  
Northeast California                    California    -121.07

In [15]:
full_df.query("us_state == 'Utah'")

,area_name,us_state,path,latitude,longitude
26178,Maple Grove,Utah,Central Utah|Maple Grove,39.02045,-112.09592
26179,Axe Handle Canyon,Utah,"Central Utah|Axe Handle Canyon|Saloon, The",39.40471,-111.68157
26180,New Canyon,Utah,"Central Utah|New Canyon|Wardel Wall, The",39.34439,-111.52640
26181,New Canyon,Utah,Central Utah|New Canyon|Jacob's Playpen,39.34408,-111.52707
26182,Log Canyon,Utah,Central Utah|Log Canyon|Schoolhouse Rock,39.64935,-111.70181
...,...,...,...,...,...
28929,Price Canyon,Utah,Central Utah|Price Canyon|Trackside Boulders|D...,39.76478,-110.89947
28930,Price Canyon,Utah,Central Utah|Price Canyon|Trackside Boulders|C...,39.76469,-110.90015
28931,Price Canyon,Utah,Central Utah|Price Canyon|Trackside Boulders|G...,39.76496,-110.89934
28932,Price Canyon,Utah,Central Utah|Price Canyon|Mayan Boulders|Mayan...,39.78146,-110.92518


## Group and Prep for Saving

In [16]:
unique_areas = full_df[~full_df['area_name'].str.contains('closed', case=False)]\
    .groupby(['area_name', 'us_state'], as_index=False)[['latitude', 'longitude']].mean()
unique_areas

,area_name,us_state,latitude,longitude
0,"""Main"" Area, The",Kansas,39.337000,-96.660500
1,* Adirondack Ice & Mixed,New York,44.060464,-73.898099
2,* Frankenstein Ice,New Hampshire,44.156453,-71.371384
3,* Mount Washington,New Hampshire,44.267827,-71.302687
4,* North Conway Area,New Hampshire,44.072423,-71.170177
...,...,...,...,...
4578,s. Books Tier,New York,41.146123,-74.165500
4579,saved Pound info,New Hampshire,43.798890,-71.796669
4580,temp,Connecticut,41.364100,-72.563720
4581,"towers at 202, The",Idaho,44.267350,-114.739810


## Add Custom Areas

Check what's in there

In [17]:
full_df[full_df['area_name'].str.contains("Citadel")]

,area_name,us_state,path,latitude,longitude
16185,"Citadel, The",Colorado,"Alpine Rock|Citadel, The",39.71530,-105.91140
35480,Citadel Crag,Wyoming,Vedauwoo|Citadel Crag,41.15546,-105.32676


In [18]:
custom_rows = [
    # name, state, path, lat, lon
    ('Woodcock Cove', 'Tennessee', '', 35.337004, -85.4532359),
    ('Citadel', 'Alabama', '', 34.47383, -85.67076),
    ("Moe's Valley", "Utah", "", 37.06928, -113.62301)
]

In [19]:
combined_df = pd.concat([
    full_df, 
    pd.DataFrame(custom_rows, columns=full_df.columns)
])

combined_df['composite_name'] = combined_df['us_state'] + ' > ' + combined_df['area_name']

combined_df.shape

(35879, 6)

In [20]:
final_locations = combined_df\
    .groupby(['composite_name'], as_index=False)\
    [['latitude', 'longitude']].mean()

final_locations.shape

(4602, 3)

In [21]:
final_locations.head()

,composite_name,latitude,longitude
0,Alabama > BankHead Forest/ Thompson Creek Trail,33.760900,-86.879900
1,Alabama > Bankhead Forest/Ship Rock,33.760900,-86.879900
2,Alabama > Bishop Mountain,34.453295,-86.379329
3,Alabama > Buck’s Pocket,34.474930,-86.060070
4,Alabama > Cherokee Bluffs,32.670950,-85.907650


In [22]:
final_locations.to_csv('climbing_locations.csv', index=False)

Let's find some defaults

In [23]:
final_locations[final_locations['composite_name'].str.contains('middle', case=False)]

,composite_name,latitude,longitude
577,Colorado > Middle Fork Needles,37.267490,-107.015100
1160,Idaho > Middle Fork of the Salmon,45.219750,-114.683910
1769,Maryland > Middle Earth,39.261610,-77.394600
1951,Massachusetts > Middlesex Fells,42.432106,-71.093075
1960,Massachusetts > Nauset MiddleSchool Boulder,41.781430,-69.989640
2254,"Missouri > Lighthouse (middle rock), The",38.113460,-93.372460
3391,Pennsylvania > Middle Wall,40.075850,-79.332820
3612,Tennessee > Middle Creek,35.144346,-85.356173
4440,Wyoming > Middle Teton,43.729920,-110.811320
